In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import tweepy
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pranjubindu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pranjubindu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df=pd.read_csv('smallTrain.csv',encoding='Latin-1')

In [3]:
df.head(20)

,tweet,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
5,http://www.dothebouncy.com/smf - some shameles...,neutral
6,2am feedings for the baby are fun when he is a...,positive
7,Soooo high,neutral
8,Both of you,neutral
9,Journey!? Wow... u just became cooler. hehe....,positive


In [4]:
df.isnull().sum()

tweet        1
sentiment    0
dtype: int64

In [5]:
wordnet=WordNetLemmatizer()

In [6]:
def change_text(a):
    a=str(a)
    text=re.sub('[^a-zA-Z]',' ',a)
    text=text.lower()
    text=nltk.word_tokenize(text)
    text=[wordnet.lemmatize(word) for word in text if word not in stopwords.words('english')]
    text=' '.join(text)
    return text

In [7]:
df['tweet']=df['tweet'].apply(change_text)
df.head()

,tweet,sentiment
0,responded going,neutral
1,sooo sad miss san diego,negative
2,bos bullying,negative
3,interview leave alone,negative
4,son put release already bought,negative


In [8]:
def polarity(message):
    if(message=='positive'):
        return(1)
    elif(message=='negative'):
        return(-1)
    else:
        return(0)
df['label']=df['sentiment'].apply(polarity)

In [9]:
'''cf=CountVectorizer()
X1=cf.fit_transform(df['tweet'])
#.values.astype('U'))
y=df['label']'''
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X1= vectorizer.fit_transform(df['tweet'])
y=df['label']

In [10]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv=ShuffleSplit(n_splits=10,test_size=0.2,random_state=42)
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.4, random_state=15)

In [11]:
print(cross_val_score(MultinomialNB(),X1,y,cv=cv,scoring='accuracy'))

[0.6196107  0.61870111 0.62033837 0.62543205 0.63034382 0.61160633
 0.61797344 0.61797344 0.61197017 0.6248863 ]


In [12]:
print(cross_val_score(MultinomialNB(),X1,y,cv=cv,scoring='accuracy').mean())

0.6198835728579225


In [38]:
x_train,x_test,y_train,y_test=train_test_split(X1,y,test_size=0.3,random_state=42)

nb=MultinomialNB()

nb.fit(x_train,y_train)

MultinomialNB()

In [14]:
prediction=nb.predict(x_test)

In [15]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [16]:
confusion_matrix(y_test,prediction)

array([[ 922, 1294,  122],
       [ 208, 2751,  412],
       [  43, 1022, 1471]], dtype=int64)

In [17]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

          -1       0.79      0.39      0.53      2338
           0       0.54      0.82      0.65      3371
           1       0.73      0.58      0.65      2536

    accuracy                           0.62      8245
   macro avg       0.69      0.60      0.61      8245
weighted avg       0.67      0.62      0.61      8245



In [18]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,prediction))

0.6238932686476653


In [19]:
accessToken="1272053768067010562-u4eunjqdSTtiE68q2J0x2NceCY4Jwx"
accessTokenSecret="8FZbdPVlqW1T5LhL2blrXzJgH9PXcty6wvHW0NFRqiHTR"
consumerKey="F0w2fjOVn4EUAH0MIjhA6lbDY"
consumerKeySecret="Bz5BJwUxx6GlSA7AtENuoXPAfV16UkMTSBngquicAh9PyEuBAX"

In [20]:
auth=tweepy.OAuthHandler(consumerKey,consumerKeySecret)
auth.set_access_token(accessToken,accessTokenSecret)
api=tweepy.API(auth)

In [21]:
def cleanText(text):
    text=re.sub('https?://[^\s]+','URL',text)
    text=re.sub('@[^\s]+','',text)
    text=re.sub(r'#','',text)
    text=re.sub(r'RT[\s]','',text)
    text=re.sub('\n',"",text)
    return text

In [22]:
numOfTweets= int(input('How many tweets to analyze'))
searchWord=input("Topic:")
date_since="2020-03-10"
tweets=tweepy.Cursor(api.search, q=searchWord, lang='en',since=date_since,tweet_mode="extended").items(numOfTweets)

How many tweets to analyze20
Topic:exams


In [23]:
df_user_tweets=pd.DataFrame([tweet.full_text for tweet in tweets],columns=['Tweets'])
df_user_tweets['Tweets']=df_user_tweets['Tweets'].apply(cleanText)
print(df_user_tweets)

                                               Tweets
0    So my wife has completed her exams.It’s been ...
1    Aaditya Thackeray Moves Supreme Court Against...
2    Whichever govt is planning to conduct exams D...
3    eye exams feel so personal like are we gonna ...
4    Basically this. Brilliantly written article b...
5   One sem of exam will be determinative of the i...
6    Respected  Sir The cases are increasing rando...
7     Pls ask d government 2 give lisence 2 d 2000...
8    Compartment exam will not held on the epidemi...
9    eye exams feel so personal like are we gonna ...
10   Omg I have to read this - I was taught rate p...
11   Sir,we are suffering from many problems.clear...
12   Twenty years ago this week, the Chunin Select...
13    🛑Our one year is being wasted in front of ev...
14   Yet another victory for Odisha Govt. decided ...
15   Respected government officials current scenar...
16   Sharing this inspiring story of a 50-year-old...
17   Madhwa girl secures 1st

In [26]:
def predict(message):
    if(nb.predict(vectorizer.transform([change_text(message)]))==1):
        return 'Positive'
    elif(nb.predict(vectorizer.transform([change_text(message)]))==0):
        return 'neutral'
    else:
        return 'Negavite'

In [27]:
df_user_tweets['Analysis']=df_user_tweets['Tweets'].apply(predict)

In [28]:
df_user_tweets.head(20)

,Tweets,Analysis
0,So my wife has completed her exams.It’s been ...,neutral
1,Aaditya Thackeray Moves Supreme Court Against...,neutral
2,Whichever govt is planning to conduct exams D...,neutral
3,eye exams feel so personal like are we gonna ...,Negavite
4,Basically this. Brilliantly written article b...,neutral
5,One sem of exam will be determinative of the i...,neutral
6,Respected Sir The cases are increasing rando...,neutral
7,Pls ask d government 2 give lisence 2 d 2000...,neutral
8,Compartment exam will not held on the epidemi...,neutral
9,eye exams feel so personal like are we gonna ...,Negavite


In [30]:
nb.predict(vectorizer.transform([change_text("it is worst movie")]))

array([-1], dtype=int64)

In [31]:
nb.predict(vectorizer.transform([change_text("carolyn cooper | ugly, poor, ignorant and black!  ")]))

array([0], dtype=int64)

In [32]:
nb.predict(vectorizer.transform([change_text("i am ugly")]))

array([-1], dtype=int64)

In [33]:
nb.predict(vectorizer.transform([change_text("i am super")]))

array([1], dtype=int64)

In [34]:
nb.predict(vectorizer.transform([change_text("i am alone")]))

array([0], dtype=int64)

In [35]:
nb.predict(vectorizer.transform([change_text("i am wonderful")]))

array([1], dtype=int64)

In [36]:
nb.predict(vectorizer.transform([change_text("hi")]))

array([0], dtype=int64)

In [37]:
nb.predict(vectorizer.transform([change_text("is it pen stand?")]))

array([0], dtype=int64)